# Student Statistics TP 4.1 - OULAD Project


## We have chosen the code module : "FFF"

## 4.1.1 Getting Started

In [2]:
code_module = "FFF"

We import the SparkSession variable and getting the app inside the Session 

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Spark SQL Stats").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

We install pandas in case it is not yet avaible inside our librairies

In [8]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd

## 4.1.2 Computing some statistics


### 1. Loading table 'studentInfo' into Spark DataFrame

In [10]:
datas = pd.read_csv("anonymisedData/studentInfo.csv")

df = spark.createDataFrame(datas)
df.show()

+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|code_module|code_presentation|id_student|gender|              region|   highest_education|imd_band|age_band|num_of_prev_attempts|studied_credits|disability|final_result|
+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|        AAA|            2013J|     11391|     M| East Anglian Region|    HE Qualification| 90-100%|    55<=|                   0|            240|         N|        Pass|
|        AAA|            2013J|     28400|     F|            Scotland|    HE Qualification|  20-30%|   35-55|                   0|             60|         N|        Pass|
|        AAA|            2013J|     30268|     F|North Western Region|A Level or Equiva...|  30-40%|   35-55|                   0|             60

### 2. Select all students that have our selected 'code_module' which is "FFF"

In [11]:
df_filtered = df.filter(df.code_module == code_module)
df_filtered.show()

+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|code_module|code_presentation|id_student|gender|              region|   highest_education|imd_band|age_band|num_of_prev_attempts|studied_credits|disability|final_result|
+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|        FFF|            2013B|     33915|     F|   South East Region|A Level or Equiva...|  30-40%|    0-35|                   0|             60|         N| Distinction|
|        FFF|            2013B|     35544|     M|               Wales|A Level or Equiva...|  80-90%|   35-55|                   0|             60|         N|        Pass|
|        FFF|            2013B|     35860|     M|   South West Region|A Level or Equiva...|   10-20|    0-35|                   0|             60

### 3.How many students have your code_module in total?

In [12]:
# Get the number of rows in the DataFrame
row_count = df_filtered.count()
print(row_count)

7762


### 4.How many students have passed (final_result == Pass) in total?

In [14]:
# Get ths number of students which Final Results are "Pass"
pass_count = df_filtered.filter(df_filtered.final_result == "Pass").count()
print(pass_count)

2978


### 5.How many students have passed (final_result == Pass) in the 2014B group (code_presentation == 2014B)?

In [15]:
# Get ths number of students which code presentation is "2014B" and Final Results are "Pass"
code_2014B_pass_count = df_filtered.filter((df_filtered['code_presentation'] == "2014B") & (df_filtered['final_result'] == "Pass")).count()
print(code_2014B_pass_count)

547


### 6.How many students have passed (final_result == Pass) in the 2014J group (code_presentation == 2014J)?

In [17]:
# Get ths number of students which code presentation is "2014J" and Final Results are "Pass"
code_2014J_pass_count = df_filtered.filter((df_filtered['code_presentation'] == "2014J") & (df_filtered['final_result'] == "Pass")).count()
print(code_2014J_pass_count)

859


### 7. What is the average studied_credits of male students (gender == M) and female students (gender == F)
### in the 2014J group (code_presentation == 2014J) 
### that have a HE Qualification or an A level or equivalent (highest_education == HE Qualification or highest_education == A level or Equivalent)

In [21]:
# What is the average studied_credits of male students (gender == M) 
# and female students (gender == F) in the 2014J group (code_presentation == 2014J) 
#that have a HE Qualification or an A level or equivalent (highest_education == HE Qualification or highest_education == A level or Equivalent)

avg_studied_cred = df_filtered.select("gender","code_presentation","highest_education","studied_credits").filter( 
    (df_filtered['code_presentation'] == '2014J')
    & ((df_filtered['highest_education'] == 'HE Qualification') | (df_filtered['highest_education'] == 'A Level or Equivalent'))).groupBy("gender").avg()

avg_studied_cred.show()

+------+--------------------+
|gender|avg(studied_credits)|
+------+--------------------+
|     F|   96.71641791044776|
|     M|    91.3239573229874|
+------+--------------------+



### 8.Replace the age_band column with the minimum value ("0-35" => 0, "35-55" => 35, "55<=" => 55) and compute it's average.

In [37]:
#Replace the age_band column with the minimum value ("0-35" => 0, "35-55" => 35, "55<=" => 55) and compute it's average.
from pyspark.sql.functions import when
from pyspark.sql.functions import avg

# Replace values in the 'age_band' column
df_filtered_age = df_filtered.withColumn('age_band', 
    when(df_filtered['age_band'] == '0-35', 0)
    .when(df_filtered['age_band'] == '35-55', 35)
    .when(df_filtered['age_band'] == '55<=', 55)
    .otherwise(df_filtered['age_band'])
)

average_age = df_filtered_age.agg(avg("age_band")).collect()[0][0]

average_age

9.167096109250194

### 9.Compute the repartition of final_result by gender, then by highest_education.

In [38]:
# Compute the repartition of final_result by gender, then by highest_education.
# Import required functions
from pyspark.sql.functions import count

# Group by 'gender' and pivot on 'final_result'
result_repartition_gender = df_filtered.groupBy('gender') \
    .pivot('final_result') \
    .agg(count('*'))

# Group by 'highest_education' and pivot on 'final_result'
result_repartition_education = df_filtered.groupBy('highest_education') \
    .pivot('final_result') \
    .agg(count('*'))

# Show the result
result_repartition_gender.show()
result_repartition_education.show()

+------+-----------+----+----+---------+
|gender|Distinction|Fail|Pass|Withdrawn|
+------+-----------+----+----+---------+
|     F|        148| 252| 540|      474|
|     M|        522|1459|2438|     1929|
+------+-----------+----+----+---------+

+--------------------+-----------+----+----+---------+
|   highest_education|Distinction|Fail|Pass|Withdrawn|
+--------------------+-----------+----+----+---------+
|A Level or Equiva...|        357| 665|1391|      902|
|  Lower Than A Level|        163| 829|1068|     1211|
|     No Formal quals|          4|  25|  20|       32|
|Post Graduate Qua...|         12|  12|  25|       14|
|    HE Qualification|        134| 180| 474|      244|
+--------------------+-----------+----+----+---------+



## 4.1.3 Statistics with Joins


### Loading datas from CSV and give previews to them

Loading "studentVle.csv" using pandas and already filter it by code_module ("FFF") to delete unnecessary rows 

In [39]:
datas_student_vle = pd.read_csv("anonymisedData/studentVle.csv")

datas_student_vle = datas_student_vle[datas_student_vle['code_module'] == code_module]
datas_student_vle

,code_module,code_presentation,id_student,id_site,date,sum_click
6253608,FFF,2013B,247666,527219,-18,2
6253609,FFF,2013B,247666,527134,-18,2
6253610,FFF,2013B,247666,526735,-18,4
6253611,FFF,2013B,247666,526835,-18,1
6253612,FFF,2013B,247666,527169,-18,2
...,...,...,...,...,...,...
10268102,FFF,2014J,1995177,882537,269,1
10268103,FFF,2014J,2143312,882547,269,3
10268104,FFF,2014J,2143312,882537,269,1
10268105,FFF,2014J,2143312,882547,269,5


Loading it as a Spark DataFrame (maybe taking some time... as it is pretty heavy :/ )

In [40]:
df_student_vle = spark.createDataFrame(datas_student_vle)
df_student_vle.show()

+-----------+-----------------+----------+-------+----+---------+
|code_module|code_presentation|id_student|id_site|date|sum_click|
+-----------+-----------------+----------+-------+----+---------+
|        FFF|            2013B|    247666| 527219| -18|        2|
|        FFF|            2013B|    247666| 527134| -18|        2|
|        FFF|            2013B|    247666| 526735| -18|        4|
|        FFF|            2013B|    247666| 526835| -18|        1|
|        FFF|            2013B|    247666| 527169| -18|        2|
|        FFF|            2013B|    247666| 527115| -18|        1|
|        FFF|            2013B|    247666| 527156| -18|        2|
|        FFF|            2013B|    247666| 527231| -18|        1|
|        FFF|            2013B|    247666| 526721| -18|       11|
|        FFF|            2013B|    247666| 527203| -18|        1|
|        FFF|            2013B|    247666| 527204| -18|        1|
|        FFF|            2013B|    247666| 526820| -18|        3|
|        F

### 1.Compute the total sum_click by region that have your selected code_module (studentVle & studentInfo).

In [41]:
#Compute the total sum_click by region that have your selected code_module (studentVle & studentInfo).

# Perform a join operation between students Info filtered by code module ("FFF") and students_vle
joined_filtered_and_stud_vle = df_filtered.select("region","id_student").join(df_student_vle, df_filtered['id_student'] == df_student_vle['id_student'], how='inner') \
.select("sum_click","region").groupBy("region").sum()

# Show the resulting DataFrame
joined_filtered_and_stud_vle.show()

+--------------------+--------------+
|              region|sum(sum_click)|
+--------------------+--------------+
| East Anglian Region|       1751577|
|   South West Region|       1226094|
|               Wales|        993038|
|West Midlands Region|       1324162|
|   South East Region|       1249517|
|North Western Region|       1564495|
|East Midlands Region|       1171843|
|             Ireland|        924807|
|    Yorkshire Region|       1050528|
|        North Region|       1331269|
|       London Region|       1958336|
|        South Region|       1933839|
|            Scotland|       1942625|
+--------------------+--------------+



Loading "vle.csv" using pandas and already filter it by code_module ("FFF") to delete unnecessary rows

In [43]:
datas_vle = pd.read_csv("anonymisedData/vle.csv")
datas_vle = datas_vle[datas_vle['code_module'] == code_module]
datas_vle

,id_site,code_module,code_presentation,activity_type,week_from,week_to
4030,527494,FFF,2013B,resource,NaN,NaN
4031,527181,FFF,2013B,subpage,NaN,NaN
4032,526812,FFF,2013B,oucontent,NaN,NaN
4033,527352,FFF,2013B,url,1.0,1.0
4034,527216,FFF,2013B,subpage,NaN,NaN
...,...,...,...,...,...,...
5992,883231,FFF,2014J,url,23.0,23.0
5993,882639,FFF,2014J,oucontent,NaN,NaN
5994,883051,FFF,2014J,subpage,NaN,NaN
5995,883264,FFF,2014J,url,29.0,29.0


Loading it as a Spark DataFrame like we did before 

In [44]:
df_vle = spark.createDataFrame(datas_vle)
df_vle.show()

+-------+-----------+-----------------+-------------+---------+-------+
|id_site|code_module|code_presentation|activity_type|week_from|week_to|
+-------+-----------+-----------------+-------------+---------+-------+
| 527494|        FFF|            2013B|     resource|      NaN|    NaN|
| 527181|        FFF|            2013B|      subpage|      NaN|    NaN|
| 526812|        FFF|            2013B|    oucontent|      NaN|    NaN|
| 527352|        FFF|            2013B|          url|      1.0|    1.0|
| 527216|        FFF|            2013B|      subpage|      NaN|    NaN|
| 527386|        FFF|            2013B|     resource|      NaN|    NaN|
| 527253|        FFF|            2013B|          url|      NaN|    NaN|
| 527427|        FFF|            2013B|     resource|      NaN|    NaN|
| 527109|        FFF|            2013B|         quiz|      NaN|    NaN|
| 526745|        FFF|            2013B|    oucontent|      NaN|    NaN|
| 527285|        FFF|            2013B|          url|      NaN| 

### 2.List distinct activity_types accessed by students in the course (studentVle & vle).


In [45]:
#List distinct activity_types accessed by students in the course (studentVle & vle).
joined_stud_vle_and_vle = df_vle.join(df_student_vle, df_vle['id_site'] == df_student_vle['id_site'], how='inner')

dist_act = joined_stud_vle_and_vle.select('activity_type').distinct()
dist_act.show()

+--------------+
| activity_type|
+--------------+
|        folder|
|          quiz|
|  ouelluminate|
|          page|
| oucollaborate|
|      resource|
|     oucontent|
|      glossary|
| questionnaire|
|           url|
|      homepage|
|        ouwiki|
|       subpage|
|repeatactivity|
|      dataplus|
|       forumng|
|      dualpane|
|  htmlactivity|
+--------------+



### 3.Compute the total sum_click by distinct activity_types performed by students that have your selected code_module (studentInfo & studentVle & vle).


We don't need to filter by code_module as we already cleaned our datas when loading it from csv using pandas .

We also don't even need "studentInfo" to perform this Operation

In [46]:
#Compute the total sum_click by distinct activity_types performed by students that have your selected code_module (studentInfo & studentVle & vle).

tot_click_act = joined_stud_vle_and_vle.select('sum_click','id_student','activity_type').groupBy('activity_type').sum('sum_click')
tot_click_act.show()


+--------------+--------------+
| activity_type|sum(sum_click)|
+--------------+--------------+
|        folder|          5420|
|          quiz|       3831026|
|  ouelluminate|         24030|
|          page|         57028|
| oucollaborate|         25788|
|      resource|        237075|
|     oucontent|       6567216|
|      glossary|          6142|
| questionnaire|         58034|
|           url|        168040|
|      homepage|       2418335|
|        ouwiki|        160706|
|       subpage|       1607759|
|repeatactivity|             9|
|      dataplus|         44070|
|       forumng|       2358160|
|      dualpane|         15845|
|  htmlactivity|          9239|
+--------------+--------------+



Loading "studentRegistration.csv" and clean it to only have the preferred code_module ("FFF")

In [47]:
# Compute the mean TMA (assessment_type) assessment score by gender for students that registered (date_registration) after day -3
# and unregistered (date_unregistration) before day 160. (studentInfo & studentRegistration & studentAssessment & assessments)

datas_stud_reg = pd.read_csv("anonymisedData/studentRegistration.csv")

datas_stud_reg = datas_stud_reg[datas_stud_reg['code_module'] == code_module]
datas_stud_reg

,code_module,code_presentation,id_student,date_registration,date_unregistration
22297,FFF,2013B,33915,-64.0,NaN
22298,FFF,2013B,35544,-22.0,NaN
22299,FFF,2013B,35860,-60.0,NaN
22300,FFF,2013B,36385,-73.0,79.0
22301,FFF,2013B,36842,-24.0,-18.0
...,...,...,...,...,...
30054,FFF,2014J,2687012,-78.0,-12.0
30055,FFF,2014J,2693932,-72.0,NaN
30056,FFF,2014J,2694139,-81.0,137.0
30057,FFF,2014J,2697921,-32.0,NaN


Loading "assessments.csv" and clean it to only have the preferred code_module ("FFF")

In [48]:
datas_asses = pd.read_csv("anonymisedData/assessments.csv")

datas_asses = datas_asses[datas_asses['code_module'] == code_module]
datas_asses

,code_module,code_presentation,id_assessment,assessment_type,date,weight
124,FFF,2013B,34865,CMA,222.0,0.0
125,FFF,2013B,34866,CMA,222.0,0.0
126,FFF,2013B,34867,CMA,222.0,0.0
127,FFF,2013B,34868,CMA,222.0,0.0
128,FFF,2013B,34869,CMA,222.0,0.0
129,FFF,2013B,34871,CMA,222.0,0.0
130,FFF,2013B,34870,CMA,222.0,0.0
131,FFF,2013B,34860,TMA,19.0,12.5
132,FFF,2013B,34861,TMA,47.0,12.5
133,FFF,2013B,34862,TMA,89.0,25.0


Loading "studentAssessment.csv"

In [49]:
datas_stud_asses = pd.read_csv("anonymisedData/studentAssessment.csv")

datas_stud_asses

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0
...,...,...,...,...,...
173907,37443,527538,227,0,60.0
173908,37443,534672,229,0,100.0
173909,37443,546286,215,0,80.0
173910,37443,546724,230,0,100.0


Loading each of previous Datas (csv) into Dataframes

In [54]:
df_stud_reg = spark.createDataFrame(datas_stud_reg)
df_asses = spark.createDataFrame(datas_asses)
df_stud_asses = spark.createDataFrame(datas_stud_asses)

print("Students Registrations ===> ")
df_stud_reg.show()

print("Assements ===> ")
df_asses.show()

print("Students Assesments ===> ")
df_stud_asses.show()

Students Registrations ===> 
+-----------+-----------------+----------+-----------------+-------------------+
|code_module|code_presentation|id_student|date_registration|date_unregistration|
+-----------+-----------------+----------+-----------------+-------------------+
|        FFF|            2013B|     33915|            -64.0|                NaN|
|        FFF|            2013B|     35544|            -22.0|                NaN|
|        FFF|            2013B|     35860|            -60.0|                NaN|
|        FFF|            2013B|     36385|            -73.0|               79.0|
|        FFF|            2013B|     36842|            -24.0|              -18.0|
|        FFF|            2013B|     37298|            -31.0|                NaN|
|        FFF|            2013B|     42668|           -122.0|                NaN|
|        FFF|            2013B|     43011|            -31.0|               -2.0|
|        FFF|            2013B|     43273|           -108.0|                NaN|

Join assesments and students assesments on field "id_assessment"

In [55]:
joined_asses_and_stud_asses = df_asses.join(df_stud_asses, df_asses['id_assessment'] == df_stud_asses['id_assessment'], how='inner')
joined_asses_and_stud_asses.show()

+-----------+-----------------+-------------+---------------+----+------+-------------+----------+--------------+---------+-----+
|code_module|code_presentation|id_assessment|assessment_type|date|weight|id_assessment|id_student|date_submitted|is_banked|score|
+-----------+-----------------+-------------+---------------+----+------+-------------+----------+--------------+---------+-----+
|        FFF|            2013J|        34873|            TMA|19.0|  12.5|        34873|    558893|            15|        0| 94.0|
|        FFF|            2013J|        34873|            TMA|19.0|  12.5|        34873|    558970|            18|        0| 76.0|
|        FFF|            2013J|        34873|            TMA|19.0|  12.5|        34873|    559067|            17|        0| 92.0|
|        FFF|            2013J|        34873|            TMA|19.0|  12.5|        34873|    559428|            18|        0| 94.0|
|        FFF|            2013J|        34873|            TMA|19.0|  12.5|        34873|   

Join all studends info having our code_module and students registrations on field "id_student"
and getting only necessary fields which are df_stud_reg["id_student"],"gender","date_registration","date_unregistration"

In [57]:
join_filtered_and_stud_reg = df_filtered.join(df_stud_reg,df_filtered['id_student'] == df_stud_reg['id_student'], how='inner').select(df_stud_reg["id_student"],"gender","date_registration","date_unregistration") 
join_filtered_and_stud_reg.show()

+----------+------+-----------------+-------------------+
|id_student|gender|date_registration|date_unregistration|
+----------+------+-----------------+-------------------+
|     96729|     M|            -44.0|                NaN|
|    191998|     M|            -29.0|               51.0|
|    193834|     M|           -121.0|                NaN|
|    505002|     M|            -46.0|                NaN|
|    553240|     F|            -20.0|               -6.0|
|    558772|     F|            -24.0|                NaN|
|   2514431|     M|            -50.0|                NaN|
|    274443|     M|           -142.0|                NaN|
|    330669|     F|            -94.0|              -78.0|
|    330669|     F|            -94.0|              -78.0|
|    417135|     F|           -138.0|                NaN|
|    467685|     M|            -38.0|                NaN|
|    508388|     M|           -136.0|                NaN|
|    568998|     M|           -121.0|               34.0|
|    568998|  

### 4.Compute the mean TMA (assessment_type) assessment score by gender for students that registered (date_registration) after day -3
### and unregistered (date_unregistration) before day 160. (studentInfo & studentRegistration & studentAssessment & assessments)

"Joining all students info filtered by our code module and students registration" with "the joined assement with students assessments" by 'id_student'
and getting only date_registration superior to -3 and date_unregistration inferior to 160

In [58]:
join_final = join_filtered_and_stud_reg.join(joined_asses_and_stud_asses,join_filtered_and_stud_reg['id_student'] == joined_asses_and_stud_asses['id_student'], how='inner') \
.filter((join_filtered_and_stud_reg.date_registration > -3) & (join_filtered_and_stud_reg.date_unregistration < 160 )) \
.select("gender","score").groupBy("gender").avg("score")
join_final.show()

+------+-----------------+
|gender|       avg(score)|
+------+-----------------+
|     F|55.69230769230769|
|     M|69.88888888888889|
+------+-----------------+



## 4.1.4 Prediction

In [30]:
#change final result


df_filtered_predict = df_filtered.withColumn('highest_education', 
    when(df_filtered['highest_education'] == 'A Level or Equivalent', 40)
    .when(df_filtered['highest_education'] == 'Lower Than A Level', 30)
    .when(df_filtered['highest_education'] == 'No Formal quals', 20)
    .when(df_filtered['highest_education'] == 'Post Graduate Qualification', 15)
    .when(df_filtered['highest_education'] == 'HE Qualification', 45)
    .otherwise(2)
)
df_filtered_predict = df_filtered_predict.withColumn('age_band', 
    when(df_filtered_predict['age_band'] == '0-35', 50)
    .when(df_filtered_predict['age_band'] == '35-55', 25)
    .when(df_filtered_predict['age_band'] == '55<=', 10)
    .otherwise(25)
)

df_filtered_predict = df_filtered_predict.withColumn('region', 
    when(df_filtered_predict['region'] == 'East Anglian Region', 1)
    .when(df_filtered_predict['region'] == 'East Midlands Region', 2)
    .when(df_filtered_predict['region'] == 'Ireland', 3)
    .when(df_filtered_predict['region'] == 'London Region', 4)
    .when(df_filtered_predict['region'] == 'North Region', 4)
    .when(df_filtered_predict['region'] == 'North Western Region', 5)
    .when(df_filtered_predict['region'] == 'Scotland', 6)
    .when(df_filtered_predict['region'] == 'South East Region', 7)
    .when(df_filtered_predict['region'] == 'South Region', 9)
    .when(df_filtered_predict['region'] == 'South West Region', 10)
    .when(df_filtered_predict['region'] == 'West Midlands Region', 6)
    .when(df_filtered_predict['region'] == 'Wales', 7)
    .when(df_filtered_predict['region'] == 'Yorkshire Region', 8)
                                                     
    .otherwise(9)
)


df_filtered_predict = df_filtered_predict.withColumn('disability', 
    when(df_filtered_predict['disability'] == 'N', 50)
    .when(df_filtered_predict['disability'] == 'Y', 0)
    .otherwise(0)
)

df_filtered_predict = df_filtered_predict.withColumn('gender', 
    when(df_filtered['gender'] == 'F', 1)
    .when(df_filtered['gender'] == 'M', 0)
    .otherwise(1)
)

df_filtered_predict = df_filtered_predict.withColumn('final_result', 
    when(df_filtered['final_result'] == 'Distinction', 20)
    .when(df_filtered['final_result'] == 'Pass', 10)
    .when(df_filtered['final_result'] == 'Fail', 5)
    .when(df_filtered['final_result'] == 'Withdrawn', 4)
    .otherwise(15)
)


df_filtered_predict = df_filtered_predict.drop('code_presentation')
df_filtered_predict = df_filtered_predict.drop('imd_band')
df_filtered_predict = df_filtered_predict.drop('code_module')
# df_filtered_predict = df_filtered_predict.drop('id_student')
df_filtered_predict = df_filtered_predict.drop('region')

# df_filtered_predict = df_filtered_predict.drop('studied_credits')
df_filtered_predict.show()

+----------+------+-----------------+--------+--------------------+---------------+----------+------------+
|id_student|gender|highest_education|age_band|num_of_prev_attempts|studied_credits|disability|final_result|
+----------+------+-----------------+--------+--------------------+---------------+----------+------------+
|     33915|     1|               40|      25|                   0|             60|        50|          20|
|     35544|     0|               40|      25|                   0|             60|        50|          10|
|     35860|     0|               40|      25|                   0|             60|        50|          10|
|     36385|     0|               30|      25|                   0|            120|         0|           4|
|     36842|     0|               30|      25|                   0|             90|        50|           4|
|     37298|     0|               45|      25|                   0|            120|        50|          10|
|     42668|     0|         

In [31]:
#random forest
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

data = spark.createDataFrame([
    (0, 7, 4, 0, 0, 60, 10, "Distinction"),
    # Add more rows of your data here
], ["gender", "region", "highest_education", "age_band", "num_of_prev_attempts", "studied_credits", "disability", "final_result"])

# Assemble features into a vector
feature_columns = df_filtered_predict.columns[:-1]  # Assuming the last column is the label

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(df_filtered_predict)

# # Split the data into training and test sets
(train_data, test_data) = data.randomSplit([0.8, 0.2])

# # Train a Random Forest model
rf = RandomForestClassifier(labelCol="final_result", featuresCol="features")
model = rf.fit(train_data)

# # Make predictions on the test data
predictions = model.transform(test_data)

# # Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="final_result", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % (accuracy))





Test Accuracy = 0.426461


In [32]:
test = joined_stud_vle_and_vle.select('sum_click','id_student','activity_type').groupBy('id_student').pivot('activity_type').agg({'sum_click': 'sum'})

test.show()
# df_filtered_predict = df_filtered_predict.groupBy('gender') \
#     .pivot('final_result') \
#     .agg(count('*'))

24/04/01 13:16:01 WARN TaskSetManager: Stage 133 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:16:05 WARN TaskSetManager: Stage 149 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+----+
|id_student|dataplus|dualpane|folder|forumng|glossary|homepage|htmlactivity|oucollaborate|oucontent|ouelluminate|ouwiki|page|questionnaire|quiz|repeatactivity|resource|subpage| url|
+----------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+----+
|    193834|    NULL|    NULL|  NULL|    230|    NULL|     395|        NULL|         NULL|     1042|           5|     4|   6|            4| 710|          NULL|      30|    206|  13|
|    587868|    NULL|    NULL|  NULL|     34|    NULL|      77|        NULL|         NULL|      515|        NULL|    21|   3|         NULL| 994|          NULL|      29|    164|  11|
|    508388|    NULL|       1|     2|    371|    NULL|     323|        NULL|            3|

In [33]:
test = test.withColumnRenamed("id_student", "id_student_test")
test.show()

24/04/01 13:16:10 WARN TaskSetManager: Stage 162 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.


+---------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+----+
|id_student_test|dataplus|dualpane|folder|forumng|glossary|homepage|htmlactivity|oucollaborate|oucontent|ouelluminate|ouwiki|page|questionnaire|quiz|repeatactivity|resource|subpage| url|
+---------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+----+
|         193834|    NULL|    NULL|  NULL|    230|    NULL|     395|        NULL|         NULL|     1042|           5|     4|   6|            4| 710|          NULL|      30|    206|  13|
|         587868|    NULL|    NULL|  NULL|     34|    NULL|      77|        NULL|         NULL|      515|        NULL|    21|   3|         NULL| 994|          NULL|      29|    164|  11|
|         508388|    NULL|       1|     2|    371|    NULL|     3

In [34]:
#test['id_student_test'] = test['id_student']

#df_filtered_pivotactivity = df_filtered_predict.join(test,df_filtered_predict['id_student'] == test['id_student_test'], how='inner')


# Perform the inner join
df_filtered_pivotactivity = df_filtered_predict.join( \
    test, \
    df_filtered_predict['id_student'] == test['id_student_test'], \
    how='inner' \
) \


df_filtered_pivotactivity = df_filtered_pivotactivity.fillna(0)
df_filtered_pivotactivity = df_filtered_pivotactivity.drop('id_student')
df_filtered_pivotactivity.show()

24/04/01 13:16:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/04/01 13:16:15 WARN TaskSetManager: Stage 176 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.


+------+-----------------+--------+--------------------+---------------+----------+------------+---------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+---+
|gender|highest_education|age_band|num_of_prev_attempts|studied_credits|disability|final_result|id_student_test|dataplus|dualpane|folder|forumng|glossary|homepage|htmlactivity|oucollaborate|oucontent|ouelluminate|ouwiki|page|questionnaire|quiz|repeatactivity|resource|subpage|url|
+------+-----------------+--------+--------------------+---------------+----------+------------+---------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+---+
|     0|               30|      25|                   3|            120|        50|          10|         193834|       0|       0|     0|    230|       0|   

In [35]:
#random forest
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

data = spark.createDataFrame([
    (0, 7, 4, 0, 0, 60, 10, "Distinction"),
    # Add more rows of your data here
], ["gender", "region", "highest_education", "age_band", "num_of_prev_attempts", "studied_credits", "disability", "final_result"])

# Assemble features into a vector
# Assuming you want to remove the seventh column
feature_columns = df_filtered_pivotactivity.columns[:6] + df_filtered_pivotactivity.columns[7:-1] # Assuming the last column is the label

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(df_filtered_pivotactivity)

# # Split the data into training and test sets
(train_data, test_data) = data.randomSplit([0.8, 0.2])

# # Train a Random Forest model
rf = RandomForestClassifier(labelCol="final_result", featuresCol="features")
model = rf.fit(train_data)

# # Make predictions on the test data
predictions = model.transform(test_data)

# # Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="final_result", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % (accuracy))





24/04/01 13:16:20 WARN TaskSetManager: Stage 192 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:16:25 WARN TaskSetManager: Stage 204 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:16:29 WARN TaskSetManager: Stage 225 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:16:35 WARN TaskSetManager: Stage 275 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:16:39 WARN TaskSetManager: Stage 287 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.


Test Accuracy = 0.653873


In [36]:
test1 = joined_asses_and_stud_asses.filter(joined_asses_and_stud_asses.date > joined_asses_and_stud_asses.date_submitted) \
.select('id_student').groupBy('id_student').count() \

test1.show()


24/04/01 13:16:44 WARN TaskSetManager: Stage 303 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.


+----------+-----+
|id_student|count|
+----------+-----+
|    568998|    3|
|    587868|    7|
|    610258|   11|
|    274443|    2|
|    417135|    3|
|    508388|   12|
|    444907|   10|
|    594229|   12|
|    646460|   11|
|    648366|   12|
|    648685|    8|
|    683902|   12|
|   2514431|    8|
|    193834|    9|
|    558772|   11|
|    400252|    7|
|    572154|    9|
|    632691|   10|
|     96729|    8|
|    505002|    2|
+----------+-----+
only showing top 20 rows



In [37]:
predict2 = df_filtered_pivotactivity.join(test1,df_filtered_pivotactivity['id_student_test'] == test1['id_student'], how='left')

predict2 = predict2.fillna(0)
predict2 = predict2.drop('id_student_test')
predict2.show()

24/04/01 13:16:44 WARN TaskSetManager: Stage 313 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:16:47 WARN TaskSetManager: Stage 315 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.


+------+-----------------+--------+--------------------+---------------+----------+------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+---+----------+-----+
|gender|highest_education|age_band|num_of_prev_attempts|studied_credits|disability|final_result|dataplus|dualpane|folder|forumng|glossary|homepage|htmlactivity|oucollaborate|oucontent|ouelluminate|ouwiki|page|questionnaire|quiz|repeatactivity|resource|subpage|url|id_student|count|
+------+-----------------+--------+--------------------+---------------+----------+------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+---+----------+-----+
|     0|               30|      25|                   3|            120|        50|          10|       0|       0|     0|    230|       0|     395|       

In [38]:
from pyspark.sql import functions as F

predict3 = joined_asses_and_stud_asses.groupBy("id_student").agg(F.sum(joined_asses_and_stud_asses["weight"] * joined_asses_and_stud_asses["score"]).alias("total_weighted_score"))
predict3.show()

24/04/01 13:16:50 WARN TaskSetManager: Stage 337 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+
|id_student|total_weighted_score|
+----------+--------------------+
|    568998|              2500.0|
|    586758|              8025.0|
|    587868|              4325.0|
|    609831|               825.0|
|    610258|              6375.0|
|   2053381|              6875.0|
|    274443|              6250.0|
|    417135|              2375.0|
|    508388|              8300.0|
|    444907|              8350.0|
|    594229|              5900.0|
|    646460|              8050.0|
|    648366|              9350.0|
|    648685|              9325.0|
|    655796|              5375.0|
|    683467|              6200.0|
|    683902|              6100.0|
|   2514431|              6975.0|
|     96729|              7850.0|
|    193834|              6100.0|
+----------+--------------------+
only showing top 20 rows



In [43]:
predict4 = predict2.join(predict3,predict2['id_student'] == predict3['id_student'], how='left')

predict4 = predict4.fillna(0)
predict4 = predict4.drop('id_student')
predict4 = predict4.drop('id_student_test')
predict4.show()

24/04/01 13:24:11 WARN TaskSetManager: Stage 381 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:24:15 WARN TaskSetManager: Stage 383 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 13:24:15 WARN TaskSetManager: Stage 385 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.


+------+-----------------+--------+--------------------+---------------+----------+------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+---+-----+--------------------+
|gender|highest_education|age_band|num_of_prev_attempts|studied_credits|disability|final_result|dataplus|dualpane|folder|forumng|glossary|homepage|htmlactivity|oucollaborate|oucontent|ouelluminate|ouwiki|page|questionnaire|quiz|repeatactivity|resource|subpage|url|count|total_weighted_score|
+------+-----------------+--------+--------------------+---------------+----------+------------+--------+--------+------+-------+--------+--------+------------+-------------+---------+------------+------+----+-------------+----+--------------+--------+-------+---+-----+--------------------+
|     0|               30|      25|                   3|            120|        50|          10|       0|       0|     0|   

In [48]:
#random forest
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Assemble features into a vector
# Assuming you want to remove the seventh column
feature_columns = predict4.columns[:6] + predict4.columns[7:-1] # Assuming the last column is the label
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(predict4)

# # Split the data into training and test sets
(train_data, test_data) = data.randomSplit([0.8, 0.2])

# # Train a Random Forest model
rf = RandomForestClassifier(labelCol="final_result", featuresCol="features")
model = rf.fit(train_data)

# # Make predictions on the test data
predictions = model.transform(test_data)

# # Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="final_result", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % (accuracy))





24/04/01 14:27:00 WARN TaskSetManager: Stage 415 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 14:27:05 WARN TaskSetManager: Stage 417 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 14:27:05 WARN TaskSetManager: Stage 419 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 14:27:07 WARN TaskSetManager: Stage 445 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 14:27:11 WARN TaskSetManager: Stage 447 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 14:27:12 WARN TaskSetManager: Stage 449 contains a task of very large size (2050 KiB). The maximum recommended task size is 1000 KiB.
24/04/01 14:27:14 WARN TaskSetManager: Stage 484 contains a task of very large size (41637 KiB). The maximum recommended task size is 1000 K

Test Accuracy = 0.631313
